matrix transformation only for perfect file (with path 0)

In [4]:
import numpy as np
import os
import pandas as pd
from scipy.optimize import minimize

# pymatgen libraries
from pymatgen.core.structure import Structure

# import function from .py file
from get_weirdos_ver3_new_cleaned import get_dx_dz_init_litype, get_sum_weirdos_Li_var_litype4

In [5]:
# Replace with your file path
litype = 4
file_perfect_poscar_48n24_wo_cif = "Li6PS5Cl_type4_trial2" # "Li6PS5Cl_type4"

activate_radius = 1
max_mapping_radius = 0.043
max_mapping_radius_48htype2 = 0.070
max_mapping_radius_48htype1_48htype2 = (max_mapping_radius + max_mapping_radius_48htype2)/2

file_ori_ref_48n24 = f"{file_perfect_poscar_48n24_wo_cif}.cif"
file_perfect_poscar_24 = "Li6PS5Cl_24_mod_2p27291.cif"
file_path_ori_ref_48n24 = f"./perfect_poscar/cif_matrix/ori/{file_ori_ref_48n24}"


In [6]:
if litype == 2:
    dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init, dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init, dx_24g_init, dz1_24g_init, dz2_24g_init = get_dx_dz_init_litype(file_path_ori_ref_48n24, litype)
elif litype == 3:
    dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init, dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init, dx_24g_init, dz1_24g_init, dz2_24g_init, dx1_48h_type3_init, dx2_48h_type3_init, dz_48h_type3_init = get_dx_dz_init_litype(file_path_ori_ref_48n24, litype)
elif litype == 4:
    dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init, dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init, dx_24g_init, dz1_24g_init, dz2_24g_init, dx1_48h_type3_init, dx2_48h_type3_init, dz_48h_type3_init, dx1_48h_type4_init, dx2_48h_type4_init, dz_48h_type4_init = get_dx_dz_init_litype(file_path_ori_ref_48n24, litype)

dx1_48h_type1, dx2_48h_type1, dz_48h_type1 = dx1_48h_type1_init, dx2_48h_type1_init, dz_48h_type1_init
dx1_48h_type2, dx2_48h_type2, dz_48h_type2 = dx1_48h_type2_init, dx2_48h_type2_init, dz_48h_type2_init
dx_24g, dz1_24g, dz2_24g = dx_24g_init, dz1_24g_init, dz2_24g_init
if litype == 3:
    dx1_48h_type3, dx2_48h_type3, dz_48h_type3 = dx1_48h_type3_init, dx2_48h_type3_init, dz_48h_type3_init
elif litype == 4:
    dx1_48h_type3, dx2_48h_type3, dz_48h_type3 = dx1_48h_type3_init, dx2_48h_type3_init, dz_48h_type3_init
    dx1_48h_type4, dx2_48h_type4, dz_48h_type4 = dx1_48h_type4_init, dx2_48h_type4_init, dz_48h_type4_init

max_mapping_radius_init = max_mapping_radius
max_mapping_radius_48htype2_init = max_mapping_radius_48htype2
max_mapping_radius_48htype1_48htype2_init = max_mapping_radius_48htype1_48htype2

In [7]:
def objective_function_var(x, max_mapping_radius, max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_ori_ref_48n24, litype, var_optitype):
    global dx1_48h_type4, dx2_48h_type4, dz_48h_type4
    dx1_48h_type4, dx2_48h_type4, dz_48h_type4 = x
    sum_weirdos_Li = get_sum_weirdos_Li_var_litype4(dx1_48h_type4, dx2_48h_type4, dz_48h_type4, max_mapping_radius, max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_ori_ref_48n24, litype, var_optitype)
    return sum_weirdos_Li

def apply_constraint(dx1_48h_type4, dz_48h_type4):
    total_48h_type4 = dx1_48h_type4 + dz_48h_type4 - 1
    return total_48h_type4

my_constraints = ({'type': 'eq', 'fun': apply_constraint})

idx_cif_file = 0
def callback_function_var(x):
    global idx_cif_file
    dx1_48h_type4, dx2_48h_type4, dz_48h_type4 = x
    sum_weirdos_Li = get_sum_weirdos_Li_var_litype4(dx1_48h_type4, dx2_48h_type4, dz_48h_type4, max_mapping_radius, max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_ori_ref_48n24, litype, var_optitype)
    print(f'{idx_cif_file:4d}   {dx1_48h_type4}   {dx2_48h_type4}   {dx1_48h_type4}   {sum_weirdos_Li}')
    idx_cif_file = idx_cif_file + 1

initial_guess_w_radius = [dx1_48h_type4_init, dx2_48h_type4_init, dz_48h_type4_init, max_mapping_radius_init, max_mapping_radius_48htype2_init]
initial_guess = [dx1_48h_type4_init, dx2_48h_type4_init, dz_48h_type4_init]

# Define bounds_w_radius for dx1 and dz (allowing up to 5 decimal places)
bounds_w_radius = [(round(dx1_48h_type4_init, 5), round(dx1_48h_type4_init, 5)),  # Lower and upper bounds_w_radius for 48h_type1
          (round(dx2_48h_type4_init, 5), round(dx2_48h_type4_init, 5)),  
          (round(dz_48h_type4_init, 5), round(dz_48h_type4_init, 5)),    
          (round(max_mapping_radius_init, 3), round(max_mapping_radius_init, 3)),                          # Lower and upper bounds_w_radius for 24g
          (round(max_mapping_radius_48htype2_init, 3), round(max_mapping_radius_48htype2_init, 3))]   

# Define bounds_w_radius for dx1 and dz (allowing up to 5 decimal places)
bounds = [(round(dx1_48h_type4_init, 5), round(dx1_48h_type4_init, 5)),  # Lower and upper bounds_w_radius for 48h_type1
          (round(dx2_48h_type4_init, 5), round(dx2_48h_type4_init, 5)),  
          (round(dz_48h_type4_init, 5), round(dz_48h_type4_init, 5))]   

# constrained_wo_radius
var_optitype = "constrained_wo_radius"
results = minimize(objective_function_var, 
                   initial_guess,
                   args = (max_mapping_radius, max_mapping_radius_48htype2, activate_radius, file_perfect_poscar_24, file_ori_ref_48n24, litype, var_optitype),
                   method = "Nelder-Mead",
                   constraints = my_constraints,
                   callback = callback_function_var)

/home/azka/.local/lib/python3.11/site-packages/scipy-1.11.0-py3.11-linux-x86_64.egg/scipy/optimize/_minimize.py:576: RuntimeWarning: Method Nelder-Mead cannot handle constraints.
  warn('Method %s cannot handle constraints.' % method,
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p+1"][0] = 0 # replace 1st element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p-1"]

   0   0.2807049999999999   0.018155666666666664   0.2807049999999999   50.0


/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p+1"][0] = 0 # replace 1st element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p-1"][(file_loc["g+p-1"]).size - 1] = 0.0 # replace last element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3749: SettingWithCopyWarning: 
A value is trying to be set o

   1   0.2807049999999999   0.018155666666666664   0.2807049999999999   50.0


/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p+1"][0] = 0 # replace 1st element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p-1"][(file_loc["g+p-1"]).size - 1] = 0.0 # replace last element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3749: SettingWithCopyWarning: 
A value is trying to be set o

...

  15   0.28488199138374476   0.018113112439986284   0.28488199138374476   43.0


/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p+1"][0] = 0 # replace 1st element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p-1"][(file_loc["g+p-1"]).size - 1] = 0.0 # replace last element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3749: SettingWithCopyWarning: 
A value is trying to be set o

  16   0.28488199138374476   0.018113112439986284   0.28488199138374476   43.0


/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p+1"][0] = 0 # replace 1st element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3747: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file_loc["g+p-1"][(file_loc["g+p-1"]).size - 1] = 0.0 # replace last element with 0
/home/azka/01_Masterthesis/01_Precondition/Masterthesis_split_litype_radius/get_weirdos_ver3_new_cleaned.py:3749: SettingWithCopyWarning: 
A value is trying to be set o

  17   0.28488199138374476   0.018113112439986284   0.28488199138374476   43.0


In [8]:
results

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 43.0
             x: [ 2.849e-01  1.811e-02  6.820e-01]
           nit: 18
          nfev: 56
 final_simplex: (array([[ 2.849e-01,  1.811e-02,  6.820e-01],
                       [ 2.849e-01,  1.811e-02,  6.820e-01],
                       [ 2.849e-01,  1.811e-02,  6.819e-01],
                       [ 2.849e-01,  1.811e-02,  6.820e-01]]), array([ 4.300e+01,  4.300e+01,  4.300e+01,  4.300e+01]))